In [1]:
#working on writing useful functions to translate/examine HEK data output

import numpy as np
from shapely.geometry import Polygon
from shapely import wkt
from sunpy.time import parse_time

In [2]:
#HEK exports certain data as unicode strings. Convert these strings to relevant, useful datatypes

def unicode2datetime(time_array):
    for i, elem in time_array:
        time_array[i] = parse_time(elem)
        
def unicode2polygon(bbox_array):
    if isinstance(bbox_array, basestring):
        bbox_array = wkt.loads(bbox_array)
    else:
        bbox_array = map(lambda x: wkt.loads(x), bbox_array)
#         for i, elem in bbox_array:
#             bbox_array[i] = wkt.loads(elem)
    return bbox_array

In [3]:
#function to determine whether 2 polygons overlap and, if so, in what manner they overlap.
#returns a list of booleans indicating different kinds of overlap 
#input parameters must both be shapely Polygons

def overlap(polygon1, polygon2):
    is_intersect = polygon1.intersects(polygon2)
    if is_intersect == False:
        is_crosses = False
        is_touches = False
        is_contained = False
        is_within = False
    else:
        is_crosses = polygon1.crosses(polygon2)
        is_touches = polygon1.touches(polygon2)
        is_contained = polygon1.contained(polygon2)
        is_within = polygon1.within(polygon2)
    return [is_intersect, is_crosses, is_touches, is_contained, is_within]

In [ ]:
#function to determine the shortest distance between two preselected points on a sphere where we know 
#the radius of the sphere
#would be of use in calculating the distance between the centers of the flux emergence and flare
#nts: maybe change x,y inputs to be tuples later
def geodesic(centroid_1, centroid_2, r):
    x1 = centroid_1.x
    x2 = centroid_2.x
    y1 = centroid_1.y
    y2 = centroid_2.y
    chord = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    s = r*np.arcsin(chord/(2*r))
    return s

In [4]:
#assign labels to events
#will have to determine how to access num without directly inputting
#keep a global var? access 
def assign_label(event_type, num):
    label = event_type + str(num)

In [5]:
#function to find shortest distance separating two areas
def short_dis_areas(polygon1_area, polygon2_area): #r? will have to add a column for flare area as that's not a given
#account for normalization by looking at disk center?


SyntaxError: invalid syntax (<ipython-input-5-4267f98e1ba1>, line 2)

In [ ]:
#function removes/sythesizes dupiclate entries for the same event

In [ ]:
#this function creates a sample data set from our merged "database" of flare events given parameters of start/stop 
#time, radial distance from center, and a minimum cutoff strength
#will have to translate flare strength classifications to numbers/something sortable

import numpy as np
import pandas as pd
import datetime
import csv

def pull_sample_flare(begin_year, begin_month, begin_day, end_year, end_month, end_day, max_dist_frm_center, min_strength):
    inputFile = 'flares_database.csv'
    flare_set = pd.read_csv(inputFile, delimiter = ',', header = 0)
    flare_set = flare_set.ix[flare_set['start_year']>=begin_year]
    flare_set = flare_set.ix[flare_set['stop_year']<=end_year]
    flare_set = flare_set.ix[flare_set['start_month']>=begin_month]
    flare_set = flare_set.ix[flare_set['stop_month']<=end_month]
    flare_set = flare_set.ix[flare_set['start_day']>=begin_day]
    flare_set = flare_set.ix[flare_set['stop_day']<=end_day]
    flare_set = flare_set.ix[flare_set['dist_frm_center']<=max_dist_frm_center]
    flare_set = flare_set.ix[flare_set['fl_peakflux']<=min_strength]
    return flare_set

# def pull_sample_flare_1(begin_time, end_time, max_dist_frm_center, min_strength):
def pull_sample_flare_1(begin_time, end_time, min_strength):
    inputFile = 'look_at_flare_data.csv'
    flare_set = pd.read_csv(inputFile, delimiter = ',', header = 0)
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
    flare_set = flare_set.ix[flare_set['event_starttime']>=begin_time]
    flare_set = flare_set.ix[flare_set['event_endtime']<=end_time]
    #flare_set = flare_set.ix[flare_set['dist_frm_center']<=max_dist_frm_center]
    flare_set = flare_set.ix[flare_set['fl_peakflux']>=min_strength]
    return flare_set

begin_time = parse_time('2014-01-01T02:34:30')
end_time = parse_time('2014-01-21T02:34:30')
max_d = 100.0
min_strength = 100.0

print pull_sample_flare_1(begin_time, end_time, min_strength)

In [ ]:
def pull_sample_ef(begin_time, end_time, delta_t):
    inputFile = 'look_at_ef_data.csv'
    ef_set = pd.read_csv(inputFile, delimiter = ',', header = 0)
    ef_set['event_starttime'] = map(parse_time, ef_set['event_starttime'])
    ef_set['event_endtime'] = map(parse_time, ef_set['event_endtime'])
    ef_set = ef_set.ix[ef_set['event_starttime']>=(begin_time-delta_t)]
    ef_set = ef_set.ix[ef_set['event_endtime']<=(end_time-delta_t)]

    return ef_set
begin_time = parse_time('2014-01-01T02:34:30')
end_time = parse_time('2014-01-21T02:34:30')
delta_t=datetime.timedelta(hours = 0, minutes = 30, seconds = 0)
print pull_sample_ef(begin_time, end_time, delta_t)

In [ ]:
def spatial_flux_check(flare_poly, ef_poly, spatial_sep, df, index):
    if geodesic(flare_poly.centroid, ef_poly.centroid)<= spatial_sep:
        return df['indentifier'][index]

In [ ]:
#function to determine whether a flare corresponds with a flux emergence

def ff(begin_year, begin_month, begin_day, end_year, end_month, end_day, max_dist_frm_center, min_strength, 
       temporal_sep_hr, temporal_sep_min, temporal_sep_sec, spatial_sep, output2file=False, out_file = None):
    delta_t = datetime.timedelta(hours = temporal_sep_hr, minutes = temporal_sep_min, seconds = temporal_sep_sec)
    begin_time = datetime.datetime(begin_year, begin_month, begin_day)
    end_time = datetime.datetime(end_year, end_month, end_day) 
    #flare_set = pull_sample_flare_1(begin_time, end_time, max_dist_frm_center, min_strength)
    flare_set = pull_sample_flare_1(begin_time, end_time, min_strength)
    ef_set = pull_sample_ef(begin_time, end_time, delta_t)
    
    is_ef = [0 for i in range(flare_set.shape[0])]
    associated_ef = []
    i = -1
    for elem in flare_set['event_starttime']:
        i += 1
        start_ef = elem-delta_t
        ef_search = ef_set.ix[ef_set['event_endtime']>=start_ef]
        ef_search = ef_search.ix[ef_search['event_endtime']<=elem]
        if ef_search.empty == False:
            fl_hpc_bbox = np.array(flare_set['hpc_bbox'])
            flare_poly = wkt.loads(fl_hpc_bbox[i])
            good_ef = []
            if ef_search.shape[0] == 1:
                ef_poly = wkt.loads(ef_search['hpc_bbox'].values[0])
                if geodesic(flare_poly.centroid, ef_poly.centroid, 966)<= spatial_sep:
                    good_ef.append(ef_search['SOL_standard'].values[0])
                else: good_ef = [0]
            else:
                ef_search['hpc_bbox'] = unicode2polygon(ef_search['hpc_bbox'])
                j = 0
                for elem in ef_search['hpc_bbox'].values:
                    if geodesic(flare_poly.centroid, elem.centroid, 966)<= spatial_sep:
                        good_ef.append(ef_search['SOL_standard'].values[j])
                    j+=1
                else: good_ef = [0]
        else: good_ef = [0]
        associated_ef.append(good_ef)
    k = 0
    for elem in associated_ef:
        if elem!=[0]:
            is_ef[k] = 1
        k+=1
    flare_set.loc[:, 'is_ef'] = is_ef
    flare_set.loc[:, 'associated_ef'] = associated_ef
    
    if output2file == True:
        if out_file == None:
            out_file = ('flare_search'+str(begin_time)+'_'+ str(end_time)+'_'+str(min_strength)+'.csv')
#             out_file = ('flare_search_'+str(begin_time)+'_'+ str(end_time)+'_'+str(max_dist_from_center)+
#                         '_'+str(min_strength)+'.csv')
        flare_set.to_csv(path_or_buf=out_file, index = False)
        
    return flare_set   
        
no = ff(2014, 1, 2, 2014, 1, 3, 100, 100, 
               0, 30, 0, 300, output2file=False, out_file = None)  